# Dynamiques multiagents

> Rappel: si la visualisation (en particulier des commandes LaTeX est défaillante), ouvrir le même fichier dans le viewer jupyter: 

Reprenons les stratégies envisagées dans le cadre du dilemme du prisonnier. 
On voit que l'on peut les classer selon deux critères: 

* le **niveau d'information** requis par la règle: on parle d'une règle de niveau $k$ lorsque les $k$ derniers coups de l'adversaire sont requis. 
* le fait que la règle soit **déterministe ou stochastique**

On peut parler d'apprentissage dès qu'une règle utilise la connaissance des coups joués par les autres joueurs, autrement dit dès qu'elle est de niveau $k\geq 1$. 

> Pouvez-vous caractériser selon ces critères les règles random, tit-for-tat, toujours trahir...





## Meilleures réponses avec plusieurs agents

On a vu dans les cours précédents la notion de meilleure réponse. 
Remarquons tout d'abord que cette notion s'étend tout à fait naturellement à plus de deux agents. 


Lorsque le système multiagent comporte plus de deux agents, on peut construire de manière similaire les tables de meilleures réponses, mais selon les possibilités de stratégies de **tous** les autres agents. 

| BR(x)  | $x_1$  | $x_2$ | $x_3$ |
|------|------|------|------|
| acheter  | emprunter| emprunter  | emprunter |
| emprunter  | acheter| acheter  | emprunter |




**Problème:** la table de meilleure réponse grandit exponentiellement avec le nombre d'agents. 


Toutefois, dans de nombreuses situations, les actions des agents n'ont que des conséquences locales. On peut donc limiter la représentation des meilleures réponses aux agents concernés. 

Imaginons par exemple que Riri, Fifi, et Loulou habitent dans une rue: Riri au début de la rue, Fifi au milieu de la rue, et Loulou à la fin de la rue. Les trois amis se demandent s'ils doivent acheter une tondeuse à gazon ou plutôt l'emprunter à leur voisin. Cette décision dépend du choix de leur voisin. Riri est voisin de Fifi, Fifi est voisin de Riri et Loulou, et Loulou n'est voisin que de Riri. 

In [4]:
def bestResponse(s,agent):
    if agent == 'riri' or agent=='loulou':
        if s['fifi']=='achete':
            return 'emprunte'
        else:
            return 'achete'
    elif agent == 'fifi':
        if s['riri']=='emprunte' and s['loulou']=='emprunte':
            return 'achete'
        else: 
            return 'emprunte'

## Dynamiques de meilleures réponses
On peut sur cette base étudier le processus dynamique qui résulte du fait que les agents jouent itérativement des meilleures réponses. 

De manière générale, on peut parler **partial best-response**: une partie de la population d'agents joue une meilleure réponse à l'état courant. On a donc deux cas limites: 

* la situation où **un seul agent** est considéré à chaque fois: les actions sont donc modifiée 
* la situation où **tous les agents** modifient leurs stratégies simultanément à chaque fois, en réponse à l'observation de l'état courant. 






Dans la version "agent unique", l'algorithme de dynamique de meilleure réponse prend donc la simple forme suivante: 

In [6]:
agents = ['riri', 'fifi', 'loulou']
s= {'riri': 'achete', 'fifi': 'achete', 'loulou': 'achete'} # vecteur de stratégies initiales
equilibrium = False
print (s)
while not equilibrium:
    equilibrium = True
    for i in agents:
        if s[i]!=bestResponse(s,i):
            s[i]=bestResponse(s,i)
            equilibrium=False
        print (s)
    
        
        
    

{'loulou': 'achete', 'riri': 'achete', 'fifi': 'achete'}
{'loulou': 'achete', 'riri': 'emprunte', 'fifi': 'achete'}
{'loulou': 'achete', 'riri': 'emprunte', 'fifi': 'emprunte'}
{'loulou': 'achete', 'riri': 'emprunte', 'fifi': 'emprunte'}
{'loulou': 'achete', 'riri': 'achete', 'fifi': 'emprunte'}
{'loulou': 'achete', 'riri': 'achete', 'fifi': 'emprunte'}
{'loulou': 'achete', 'riri': 'achete', 'fifi': 'emprunte'}
{'loulou': 'achete', 'riri': 'achete', 'fifi': 'emprunte'}
{'loulou': 'achete', 'riri': 'achete', 'fifi': 'emprunte'}
{'loulou': 'achete', 'riri': 'achete', 'fifi': 'emprunte'}


> Que se passe-t-il sur cet exemple si on considère la dynamique où tous les agents modifient simultanément leurs stratégies?  

#### Quelques propriétés simples des dynamiques de meilleures réponses: 

* Lorsqu'un équilibre est atteint, il s'agit bien d'un équilibre de Nash (par définition). 
* Les dynamiques de meilleures réponses cyclent nécessairement lorsqu'il n'y a pas d'équilibre de Nash
* Les dynamiques de meilleures réponses peuvent cycler même si il existe des équilibres de Nash 
* Toutefois, sous certaines conditions, on peut montrer qu'elles convergent nécessairement vers des équilibres de Nash. 

## Fictitious Play



Supposons maintenant que les agents apprennent plus spécifiquement le comportement des autres joueurs, et jouent la meilleure réponse à la prédiction qu'ils font du comportement des autres agents. 
Un des modèles les plus simples d'un tel type d'apprentissage a été proposé par Brown (1951) sous le nom de *fictitious play*. 

Il repose donc sur les phases suivantes: 

* prédiction des actions des autres joueurs
* choix de la meilleure réponse à la stratégie des autres agents
* observation des actions des autres joueurs, mise à jour de la prédiction

**(Standard) Fictitious Play**: 
Nous présentons ici le modèle dans un contexte où seuls deux agents sont en interaction. (Le modèle s'étend à plus d'agents, mais il faut alors faire des choix sur la manière de combiner les probabilités assigné

Chaque joueur tient à jour un modèle simple des autres joueurs (la fréquence de sélection des actions) et joue sa meilleure réponse à ce modèle probabiliste. 
Autrement dit: 


#### Exemple: 

Supposons que le joueur \textcolor{blue}{bleu} joue aveu ou silence une fois sur deux. 

La table suivante donne les gains pour le joueur R. 

| | c | d |
| ----|---- | ---- |
|**a** | 8 | 10 |
|**b** | 10 | 0 |


Supposons que la stratégie de B soit de jouer alternativement c et d. 
Il faut également que l'agent fasse une prédiction *a priori* sur l'action initiale de l'agent 


| | BR(R) | coup B | #c | #d | P(c) | P(d) |
| ----|---- | ---- | ---- | ---- | ---- |
|**a** | 8 | 10 |
|**b** | 10 | 0 |


## Autres techniques d'apprentissage

Considérons le problème 